In [1]:
from torch.utils.data import DataLoader
from torchvision import transforms
from concept_model.dataset import CUBImageToAttributes

batch_size = 32
num_workers = 2

train_preprocess = transforms.Compose(
    [
        transforms.RandomResizedCrop(299),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

training_data = CUBImageToAttributes(train=True, transform=train_preprocess)
test_data = CUBImageToAttributes(train=False)

train_dataloader = DataLoader(
    training_data, batch_size=batch_size, num_workers=num_workers, shuffle=True
)
test_dataloader = DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break


Shape of X [N, C, H, W]: torch.Size([32, 3, 299, 299])
Shape of y: torch.Size([32, 312]) torch.int64


In [2]:
import torch

model = torch.hub.load(
    "pytorch/vision:v0.10.0", "inception_v3", weights="IMAGENET1K_V1"
)
model.AuxLogits.fc = torch.nn.Linear(in_features=768, out_features=312)
model.fc = torch.nn.Linear(in_features=2048, out_features=312)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
model.to(device)

loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)


def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(torch.float).to(device)
        # Compute prediction and loss
        logits, aux_logits = model(X)
        loss = loss_fn(logits, y) + 0.4 * loss_fn(aux_logits, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    correct_using_attributes = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y.to(torch.float)).item()
            correct += (
                ((torch.sigmoid(pred) > 0.5).to(torch.int64) == y)
                .type(torch.float)
                .all(dim=1)
                .sum()
                .item()
            )
            correct_attributes = torch.sum(
                ((torch.nn.Sigmoid()(pred) > 0.5).to(torch.int64) == y).to(torch.float)
            )
            num_attributes = y.shape[1]
            correct_using_attributes += correct_attributes.item() / num_attributes
    test_loss /= num_batches
    correct /= size
    correct_using_attributes /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Accuracy (Attribute-wise): {(100*correct_using_attributes):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )
    return correct_using_attributes


Using cache found in /home/joanna/.cache/torch/hub/pytorch_vision_v0.10.0
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /home/joanna/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100.0%


Using cuda device


In [3]:
epochs = 100
max_acc = 0
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    curr_acc = test(test_dataloader, model, loss_fn)
    if curr_acc > max_acc:
        torch.save(model.state_dict(), "independent_image_to_attributes.pth")
        max_acc = curr_acc
print("Done!")


Epoch 1
-------------------------------
loss: 0.998570  [    0/ 5994]
loss: 0.489658  [ 3200/ 5994]
Test Error: 
 Accuracy: 0.0%, Accuracy (Attribute-wise): 90.3%, Avg loss: 0.281488 

Epoch 2
-------------------------------
loss: 0.420159  [    0/ 5994]
loss: 0.386683  [ 3200/ 5994]
Test Error: 
 Accuracy: 0.0%, Accuracy (Attribute-wise): 90.3%, Avg loss: 0.262413 

Epoch 3
-------------------------------
loss: 0.382060  [    0/ 5994]
loss: 0.387880  [ 3200/ 5994]
Test Error: 
 Accuracy: 0.0%, Accuracy (Attribute-wise): 90.3%, Avg loss: 0.257335 

Epoch 4
-------------------------------
loss: 0.358059  [    0/ 5994]
loss: 0.353023  [ 3200/ 5994]
Test Error: 
 Accuracy: 0.0%, Accuracy (Attribute-wise): 90.4%, Avg loss: 0.253840 

Epoch 5
-------------------------------
loss: 0.360708  [    0/ 5994]
loss: 0.369440  [ 3200/ 5994]
Test Error: 
 Accuracy: 0.0%, Accuracy (Attribute-wise): 90.5%, Avg loss: 0.250520 

Epoch 6
-------------------------------
loss: 0.348521  [    0/ 5994]
loss: